In [1]:
from robot_descriptions.a1_description import URDF_PATH
from darli.backend import PinocchioBackend, CasadiBackend, JointType
from darli.modeling import Robot, Functional
from darli.modeling.integrators import ForwardEuler, RK4
import numpy as np

In [2]:
model = Functional(CasadiBackend(URDF_PATH, root_joint=JointType.FREE_FLYER))
model.update_selector(passive_joints=range(6))

In [3]:
np.random.seed(0)
tau = np.random.randn(model.nu)

tau

array([ 1.76405235,  0.40015721,  0.97873798,  2.2408932 ,  1.86755799,
       -0.97727788,  0.95008842, -0.15135721, -0.10321885,  0.4105985 ,
        0.14404357,  1.45427351])

In [4]:
q0 = np.array([0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
v0 = np.zeros(model.nv)

x0 = np.concatenate([q0, v0])

x0.shape, model.nq, model.nv

((37,), 19, 18)

## Functional way

In [5]:
euler_roll = model.state_space.rollout(
    dt=2e-3, n_steps=30, control_sampling=4e-3, integrator=ForwardEuler
)

euler_roll

Function(rollout:(state[37],controls[12x15])->(next_state[37]) SXFunction)

In [6]:
%timeit euler_roll()

1.45 ms ± 193 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [7]:
rk4_roll = model.state_space.rollout(
    dt=2e-3, n_steps=30, control_sampling=4e-3, integrator=RK4
)

rk4_roll

Function(rollout:(state[37],controls[12x15])->(next_state[37]) SXFunction)

In [8]:
%timeit rk4_roll()

5.55 ms ± 151 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


## Pinocchio way

In [9]:
model = Robot(PinocchioBackend(URDF_PATH, root_joint=JointType.FREE_FLYER))
model.update_selector(passive_joints=range(6))

In [10]:
q0 = np.array([0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
v0 = np.zeros(model.nv)

x0 = np.concatenate([q0, v0])

np.random.seed(0)
tau = np.random.randn(model.nu, 15)


model.state_space.rollout(x0, tau, dt=2e-3, control_sampling=4e-3, n_steps=30)

array([ 3.76576015e-04, -5.90776756e-04, -1.86484692e-02, -3.55866284e-02,
        2.98828478e-04, -2.06836938e-03,  9.99364410e-01,  1.20123327e-01,
        8.94504016e-02, -2.04843323e-01,  5.97518327e-02, -2.64078009e-02,
       -1.08357083e-02,  9.61331223e-02,  9.85161233e-02, -1.30548863e-01,
        8.77269278e-02, -3.11990450e-02,  7.02278288e-02,  1.64054750e-02,
        2.85389016e-02, -6.01619857e-01, -1.77246306e+00, -3.42885311e-02,
       -1.46697075e-01,  2.98211623e+00,  3.88621024e+00, -8.52350431e+00,
        1.48212236e+00, -2.21680733e+00,  4.25183734e+00,  2.29552203e+00,
        3.49743072e+00, -5.23065688e+00,  2.12015156e+00,  1.95744729e-01,
       -6.94286048e-01])

In [11]:
%timeit model.state_space.rollout(x0, tau, dt=2e-3, control_sampling=4e-3, n_steps=30, integrator=ForwardEuler)

1.49 ms ± 180 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [12]:
euler_state = model.state_space.rollout(
    x0, tau, dt=2e-3, control_sampling=4e-3, n_steps=30, integrator=ForwardEuler
)

euler_state, "quaternion norm", np.linalg.norm(euler_state[3:7])

(array([ 3.76576015e-04, -5.90776756e-04, -1.86484692e-02, -3.55866284e-02,
         2.98828478e-04, -2.06836938e-03,  9.99364410e-01,  1.20123327e-01,
         8.94504016e-02, -2.04843323e-01,  5.97518327e-02, -2.64078009e-02,
        -1.08357083e-02,  9.61331223e-02,  9.85161233e-02, -1.30548863e-01,
         8.77269278e-02, -3.11990450e-02,  7.02278288e-02,  1.64054750e-02,
         2.85389016e-02, -6.01619857e-01, -1.77246306e+00, -3.42885311e-02,
        -1.46697075e-01,  2.98211623e+00,  3.88621024e+00, -8.52350431e+00,
         1.48212236e+00, -2.21680733e+00,  4.25183734e+00,  2.29552203e+00,
         3.49743072e+00, -5.23065688e+00,  2.12015156e+00,  1.95744729e-01,
        -6.94286048e-01]),
 'quaternion norm',
 1.0)

In [13]:
%timeit model.state_space.rollout(x0, tau, dt=2e-3, control_sampling=4e-3, n_steps=30, integrator=RK4)

8.31 ms ± 972 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [14]:
rk4_state = model.state_space.rollout(
    x0, tau, dt=2e-3, control_sampling=4e-3, n_steps=30, integrator=RK4
)

rk4_state, "quaternion norm", np.linalg.norm(rk4_state[3:7])

(array([ 3.28031153e-05, -2.75608771e-05, -1.19513565e-03, -1.69214588e-03,
        -5.40001291e-05, -1.48067584e-04,  9.99998556e-01,  5.87030990e-03,
         7.94249395e-03, -1.74507362e-02,  2.84304581e-03, -4.39651886e-03,
         8.46712564e-03,  4.47790182e-03,  7.01606052e-03, -1.05225893e-02,
         4.11139492e-03,  4.06005873e-04, -1.39273193e-03,  3.28216779e-05,
        -2.48331438e-05, -1.19517838e-03, -3.38427802e-03, -1.07776273e-04,
        -2.96480264e-04,  5.87030990e-03,  7.94249395e-03, -1.74507362e-02,
         2.84304581e-03, -4.39651886e-03,  8.46712564e-03,  4.47790182e-03,
         7.01606052e-03, -1.05225893e-02,  4.11139492e-03,  4.06005873e-04,
        -1.39273193e-03]),
 'quaternion norm',
 1.0)